In [2]:
from collections import Counter
def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >=min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [3]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, mmin_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n=len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    contiune
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
                
    counter = {k:v for k, v in counter.items() if v >=min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [4]:
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i,j), v in  d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape =(n_rows, n_cols))

In [5]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sen in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

In [6]:
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1
    
    #initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df)*np.ones(A.shape[0]).reshape(-1,1)
    
    #iteration
    for _ in range(max_iter):
        R = df *(A*R)+bias
        
    return R

In [7]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [8]:
from collections import Counter
from scipy.sparse import csr_matrix
import math

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    
    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(toekns):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data), (rows, cols), shape=(n_sents, n_sents))

def textrank_sent_sim(s1,s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common/base
def cosine_sent_sime(s1,s2):
    if(not s1)or(not s2):
        return 0
    
    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v**2 for v in s1.values()))
    norm2 = math.sqrt(sum(v**2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v*s2.get(k,0)
    return prod / (norm1 *norm2)

In [9]:
def textrank_keysentece(sents, tokenize, min_count, similarity, df=0.85, max_iter=30, topk=5):
    g = sent_graph(sents, tokenize, min_count, min_sim, similarity)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

In [10]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [11]:
from textrank.summarizer import KeywordSummarizer
keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
sents = [
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', 
  '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', 
  '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', 
  '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', 
  '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', 
  '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', 
  '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', 
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', 
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', 
  '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다', 
  '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다', 
  '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다', 
  '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다', 
  '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다', 
  '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다', 
  '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다', 
  '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다', 
  '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다', 
  '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다', 
  '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]
keywords = keyword_extractor.summarize(sents, topk=30)

In [12]:
keywords

[('용의자/NNP', 3.040833543583403),
 ('사제총/NNP', 2.505798518168069),
 ('성씨/NNP', 2.4254730689696298),
 ('서울/NNP', 2.399522533743009),
 ('경찰/NNG', 2.2541631612221043),
 ('오후/NNG', 2.154778397410354),
 ('폭행/NNG', 1.9019818685234693),
 ('씨/NNB', 1.7517679455874249),
 ('발사/NNG', 1.658959293729613),
 ('맞/VV', 1.618499063577056),
 ('분/NNB', 1.6164369966921637),
 ('번동/NNP', 1.4681655196749035),
 ('현장/NNG', 1.4530182347939307),
 ('시/NNB', 1.408892735491178),
 ('경찰관/NNP', 1.4012941012332316),
 ('조사/NNG', 1.4012941012332316),
 ('일/NNB', 1.3922748983755766),
 ('강북구/NNP', 1.332317291003927),
 ('연합뉴스/NNP', 1.3259099432277819),
 ('이씨/NNP', 1.2869280494707418),
 ('경위/NNG', 1.2442970385565473),
 ('소지/NNG', 1.2383874843043485),
 ('오/NNP', 1.2204773446319281),
 ('패사/NNG', 1.2204773446319281),
 ('인근/NNG', 1.2118828522182832),
 ('신고/NNG', 1.174042491690959),
 ('숨지/VV', 1.1483016731686424),
 ('김/NNP', 1.1443609482728325),
 ('있/VV', 1.134049499636939),
 ('검거/NNG', 1.1026800788933409)]

In [13]:
with open('lalaland_komoran.txt', encoding = 'utf-8') as f:
    sents = [sent.strip() for sent in f]
with open('lalaland.txt', encoding = 'utf-8')as f:
    texts = [sent.strip() for sent in f]
print(len(sents), len(texts))

15595 15595


In [14]:
from textrank import KeywordSummarizer
def komoran_tokenize(sent):
    words = sent.split()
    words = [w for w in words if('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word,rank))

영화/NNG (1.73e+02)
보/VV (1.29e+02)
좋/VA (65.5)
하/VV (52.0)
것/NNB (47.4)
같/VA (45.4)
영화/NNP (43.8)
음악/NNG (43.6)
꿈/NNG (41.4)
있/VV (40.8)
없/VA (35.9)
마지막/NNG (31.9)
수/NNB (30.1)
사랑/NNG (28.3)
아름답/VA (26.5)
현실/NNG (24.8)
되/VV (23.9)
노래/NNG (23.4)
생각/NNG (23.2)
스토리/NNP (21.4)
번/NNB (20.3)
거/NNB (19.7)
최고/NNG (19.2)
때/NNG (19.1)
사람/NNG (19.0)
여운/NNP (17.5)
뮤지컬/NNP (16.9)
나오/VV (16.5)
듯/NNB (16.1)
영상미/NNG (16.0)


In [15]:
keyword_extractor = KeywordSummarizer(
    tokenize= lambda x:x.split(),
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{}({:.3})'.format(word, rank))

ㄴ/ETM(1.24e+02)
고/EC(1.03e+02)
영화/NNG(96.8)
는/ETM(94.6)
이/VCP(92.3)
이/JKS(92.0)
하/XSV(85.2)
에/JKB(79.0)
았/EP(76.1)
보/VV(73.5)
었/EP(72.8)
다/EC(68.3)
을/JKO(64.2)
하/XSA(58.8)
의/JKG(58.4)
도/JX(52.7)
ㄹ/ETM(50.2)
가/JKS(47.2)
게/EC(46.7)
는/JX(42.3)
어/EC(37.9)
좋/VA(37.6)
를/JKO(34.3)
아/EC(33.8)
은/ETM(33.7)
들/XSN(32.6)
은/JX(32.0)
하/VV(29.8)
것/NNB(26.7)
과/JC(26.5)


In [16]:
#핵심문장 선택(min_sim이상의 유사도를 지닌 문장에 pagerank를 적용해 핵심문장 선택)
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
for idx,rank,komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

calculating textrank sentence similarity was done with 15595 sents
trained TextRank. n sentences = 15595
#5861 (6.12) : 사랑에 대해 다시 한 번 생각해 볼 수 있게 하는 영화인 것 같습니다 장면 처리도 좋았어요 여운이 많이 남는 영화입니다 꼭 보세요

#5947 (5.8) : 아 진짜 평점 처음 써본다 진짜 후회 안할 영화 나중에 다시 봐도 좋을것 같다 오프닝에서 신나는 노래부터 마지막의 상상 하는 씬까지 너무 좋음 결말이 여운이 있다고 해야하나 슬프다고 해야하나 꿈얘기 할때 현실성 있어서 눈물날뻔 결말이짱

#5076 (5.69) : 옛날 영화같은 느낌의 기법 사람을 행복하게 만드는 음악 약간의 촌스러움이 마음을 간지를 수 있는 지극히 현실적인 사랑과 꿈이야기를 로맨틱하게 풀어낸 영화로 연말 영화로 보기 좋은 것 같아요

#6665 (5.41) : 인생영화다 노래도 너무 좋고 배우 소품 배경 장면들 하나하나 맘에 안 드는게 없다 ㅠㅠ 특히 마지막 셉oo에서의 내용은 진짜 잊을 수가 없을 거같다 보고나면 먹먹하고 안타까운 느낌이 드는데 그래도 황홀하고 아름다운 영화다

#9271 (5.28) : 연출 음악 영상미 엔딩은 정말 좋았다 마지막에 남녀주인공이 나눈 눈빛이 아직도 잊혀지지않을만큼 여운이 남는 영화였다 초중반 약간 지루하긴했었다 배우들 춤연습을 많이한게보였음 꿈 성공 과 사랑을 다 가질수 없다는것을 현실적으로보여준 영화가아니었나싶다

#13909 (5.12) : 인생 최고의 영화 또보고싶다 영상미 음악 스토리 다 좋아요

#5922 (4.99) : 정말 영상미랑 음악은 최고였다 그리고 신선했다 음악이 너무 멋있어서 연기를 봐야 할지 노래를 들어야 할지 모를 정도로 그리고 보고 나서 생각 좀 많아진 영화 정말 이 연말에 보기 좋은 영화 인 것 같다

#11408 (4.88) : 진짜 그냥 좋았던 영화 두번봐도 재밌을영화

#12362 (4.88) : 보고 난 후

In [17]:
sents = [
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', 
  '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', 
  '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', 
  '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', 
  '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', 
  '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', 
  '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', 
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', 
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', 
  '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다', 
  '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다', 
  '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다', 
  '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다', 
  '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다', 
  '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다', 
  '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다', 
  '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다', 
  '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다', 
  '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다', 
  '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]

In [18]:
#띄어쓰기 기준으로 adjacent sentence graph만들기
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = lambda x:x.split(),
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기


In [19]:
#konlpy의 komoran을 이용해 토크나이징과 핵심문장을 한 번에 추출하는 예시
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기
신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다


In [20]:
#띄어쓰기 기준으로 나뉘어진 어절에서 3음절의 subwords 토크나이저
def subword_tokenizer(sent, n=3):
    def subword(token, n):
        if len(token) <= n:
            return [token]
        return[token[i:i+n] for i in range(len(token) -n + 1)]
    return [sub for token in sent.split() for sub in subword(token, n)]
subword_tokenizer('이것은 부분단어의 예시이다 짧은 어절은 그대로 나온다')
#이 방법을 이용하여도 위 문장과 뉴스는 동일하게 나옴


['이것은', '부분단', '분단어', '단어의', '예시이', '시이다', '짧은', '어절은', '그대로', '나온다']

In [21]:
#문장별 중요도(pagerank값)
summarizer.R

array([1.70101956, 0.80126597, 1.29270438, 0.86509755, 0.71388517,
       0.8847745 , 0.82988149, 1.48860432, 1.42785904, 1.37332646,
       0.98301038, 0.54650808, 0.58873251, 1.0303169 , 1.32943091,
       1.02482532, 0.713935  , 0.96237969, 0.93278054, 0.50966223])

뉴스 기사는 대부분 첫 문장이 중요하다. 실제로 위의 예시에서도 첫 문장이 가장 중요한 핵심 문장으로 선택되었다. 만약 마지막 문장이 중요하다고 가정한다면 이러한 정보를 bias에 추가하면 된다
numpy.ndarray 형태로 bias를 만든다. 마지막 문장이 다른 문장보다 10배 중요하다고 가정하고 이를 summarize함수의 bias에 입력하면 가장 먼저 맨 마지막 문장이 중요한 문장으로 선택됩니다. 다른 문장 중에서도 맨 마지막 문장과 비슷할 수록 상대적 중요도가 더 커지는 것.

In [22]:

import numpy as np
bias = np.ones(len(sents))
bias[-1] = 10

keysents = summarizer.summarize(sents, topk=3, bias=bias)
for _, _, sent in keysents:
    print(sent)

일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다


In [23]:
summarizer.R

array([1.53926495, 0.68873706, 1.15643348, 0.74610099, 0.60204057,
       0.77869673, 0.71962531, 1.31463192, 1.31044833, 1.44213027,
       0.86064789, 0.45297471, 0.46698315, 1.15465766, 1.20123797,
       0.90827787, 0.78689713, 0.85042996, 1.06684512, 1.95293892])